In [1]:
import os
from pathlib import Path

# Preparation

First of all, make sure to download the corresponding medical ontologies to build the term dictionaries for bronco.
 - Treatments: [OPS](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html). Scroll to "OPS", 
 - Medications: [ATC](https://www.wido.de/publikationen-produkte/arzneimittel-klassifikation/)
 - Diagnosis: [ICD10GM](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html)
 
In the config file for BRONCO `../conf/bronco.yaml`, modify the paths so they point the extracted ontologies.

We can already use xMEN to prepare the term dictionaries. In your terminal, navigate to the xMEN root folder and run:
 - `xmen dict conf/bronco.yaml --code dicts/atc2017_de.py --output temp/ --key atc`
 - `xmen dict conf/bronco.yaml --code dicts/ops2017.py --output temp/ --key ops`
 - `xmen dict conf/bronco.yaml --code dicts/icd10gm2017.py --output temp/ --key icd10gm`
 
Now use such dictionaries to build the indexes. For this example, we will use only SapBERT indexes and leave aside N-Gram:
 - `xmen index conf/bronco.yaml --dict temp/atc.jsonl --output temp/atc_index --sapbert`
 - `xmen index conf/bronco.yaml --dict temp/ops.jsonl --output temp/ops_index --sapbert`
 - `xmen index conf/bronco.yaml --dict temp/icd10gm.jsonl --output temp/icd10gm_index --sapbert`
 
Finally, load the adapted config file and the BRONCO150 dataset using BigBIO:

In [2]:
from xmen.confhelper import load_config
config = load_config("../conf/bronco.yaml")

import datasets
path_to_data = r"../../BRONCO150" # paste here the path to the local data
ds = datasets.load_dataset(path = "bigbio/bronco", 
                           name = "bronco_bigbio_kb", 
                           data_dir=path_to_data)

ds

Found cached dataset bronco (/dhc/home/ignacio.rodriguez/.cache/huggingface/datasets/bigbio___bronco/bronco_bigbio_kb-data_dir=..%2F..%2FBRONCO150/1.0.0/cab8fc4a62807688cb5b36df7a24eb7f364314862c4196f6ff2db3813f2fe68b)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 5
    })
})

# Candidate Generation
We will use `SapBERTLinker`, which uses a Transformer model to retrieve candidates with dense embeddings.

We could have also used `TFIDFNGramLinker` (see `xmen/notebooks/BioASQ_DisTEMIST.ipynb` for an example).

In [9]:
from notebook_util import analyze
from xmen.linkers import TFIDFNGramLinker, SapBERTLinker, EnsembleLinker
from xmen.linkers.util import filter_and_apply_threshold
from datasets import DatasetDict
from xmen.evaluation import entity_linking_error_analysis, evaluate_at_k
from xmen.linkers.util import filter_and_apply_threshold

In [4]:
embedding_model_name = 'cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR'
index_path = Path("../temp/icd10gm_index/index/sapbert")
os.path.exists(index_path)

True

In [5]:
# Clear singleton to free up memory
SapBERTLinker.clear()
sapbert_linker = SapBERTLinker(
    embedding_model_name = embedding_model_name,
    index_base_path = index_path,
    k = 1000
)

pred_sapbert = sapbert_linker.predict_batch(ds, batch_size=128)

[05/28/23 14:36:20] INFO     Loading hierarchical faiss index                                ]8;id=847863;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=230809;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/sap_bert_linker.py#148\148]8;;\

                    INFO     Loading index from                                                 ]8;id=822163;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=413600;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             ../temp/icd10gm_index/index/sapbert/embed_faiss_hier.pickle                           

[05/28/23 14:36:21] INFO     Loaded index of type <class 'faiss.swigfaiss_avx2.IndexHNSWFlat'>  ]8;id=105163;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=409492;file:///dhc/home/ignacio.rodriguez/xmen/xmen/linkers/faiss_indexer.py#66\66]8;;\
                             and size 15930                                                                        

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [11]:
pred_sapbert

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 5
    })
})

In [14]:
# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['train'].select([0]), pred_sapbert['train'].select([0]))

Perf@1 0.10778781038374718
Perf@2 0.14221218961625282
Perf@4 0.17099322799097066
Perf@8 0.20033860045146726
Perf@16 0.23306997742663657
Perf@32 0.26297968397291194
Perf@64 0.2979683972911964


In [3]:
from xmen.evaluation import entity_linking_error_analysis, evaluate_at_k
from xmen.linkers.util import filter_and_apply_threshold